In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import SVC
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np
import re
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier

In [2]:
dfe = pd.read_csv('/content/drive/MyDrive/Bracu/Fall 2022/CSE 431/SarcasDetectionEDA.csv')
df = pd.read_csv('/content/drive/MyDrive/Bracu/Fall 2022/CSE 431/SarcasDetection.csv')
df.head()

,Unnamed: 0,Comments,Label
0,0,আমি মনে করি যখন মেয়েরা উদ্দেশ্য অনুসারে বোবা ...,1.0
1,1,আমি যখন কোনও গুরুত্বপূর্ণ প্রশ্ন জিজ্ঞাসা করি ...,1.0
2,2,বাহ ... আমি দেখতে দেখতে পাচ্ছি সত্যিই দোষী দোষ...,1.0
3,3,"হাহ, মজার বিষয়।#আনোয়েড #গ্রুআপ",1.0
4,4,থান্ডার আমাকে 830 এ জেগে উঠলে এটি ভালবাসুন,1.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112 entries, 0 to 5111
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5112 non-null   int64  
 1   Comments    5112 non-null   object 
 2   Label       5112 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 119.9+ KB


In [4]:
df.describe()

,Unnamed: 0,Label
count,5112.000000,5112.000000
mean,2555.500000,0.382042
std,1475.851619,0.485934
min,0.000000,0.000000
25%,1277.750000,0.000000
50%,2555.500000,0.000000
75%,3833.250000,1.000000
max,5111.000000,1.000000


#PRE PROCESSING

In [5]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    # english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)    
    text=emoji_pattern.sub(r'', text)
    # text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
        else:
            no_punct = no_punct + " "
    return no_punct

def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [6]:
df['cleanText'] = df.Comments.apply(lambda x: preprocessing(str(x)))

In [7]:
df.head()

,Unnamed: 0,Comments,Label,cleanText
0,0,আমি মনে করি যখন মেয়েরা উদ্দেশ্য অনুসারে বোবা ...,1.0,আমি মনে করি যখন মেয়েরা উদ্দেশ্য অনুসারে বোবা ...
1,1,আমি যখন কোনও গুরুত্বপূর্ণ প্রশ্ন জিজ্ঞাসা করি ...,1.0,আমি যখন কোনও গুরুত্বপূর্ণ প্রশ্ন জিজ্ঞাসা করি ...
2,2,বাহ ... আমি দেখতে দেখতে পাচ্ছি সত্যিই দোষী দোষ...,1.0,বাহ আমি দেখতে দেখতে পাচ্ছি সত্যিই দোষী দোষ...
3,3,"হাহ, মজার বিষয়।#আনোয়েড #গ্রুআপ",1.0,হাহ মজার বিষয় আনোয়েড গ্রুআপ
4,4,থান্ডার আমাকে 830 এ জেগে উঠলে এটি ভালবাসুন,1.0,থান্ডার আমাকে এ জেগে উঠলে এটি ভালবাসুন


In [8]:
!pip install bnlp_toolkit



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 26.5 MB/s 
     |████████████████████████████████| 23.9 MB 1.2 MB/s 
     |████████████████████████████████| 1.0 MB 33.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


#STOPWORDS DONT RUN

In [9]:
from bnlp.corpus import stopwords
print(len(stopwords))

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


398


/usr/local/lib/python3.8/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
data =pd.read_excel('/content/drive/MyDrive/Bracu/Fall 2022/CSE 431/stopwords_bangla.xlsx')
stopwords2 = data['words'].tolist()
print(len(stopwords2))
stopwords.extend(stopwords2)
print(len(stopwords))

732
1130


In [11]:
def stopwordRemoval(text):    
    x=str(text)
    l=x.split()
    stm=[elem for elem in l if elem not in stopwords]
    out=' '.join(stm)
    return str(out)

In [12]:
df['cleanText'] = df.cleanText.apply(lambda x: stopwordRemoval(str(x)))

#BALANCING

In [13]:
df['Label'].value_counts()

0.0    3159
1.0    1953
Name: Label, dtype: int64

In [14]:
def dataset_balance(data):
    data_positive = data[(data['Label']== 1)]
    data_negative = data[(data['Label']==0)]
    data_positive_upsampled = resample(data_positive, replace = True, n_samples =  data['Label'].value_counts()[0], random_state = 42)
    balanced_dataset = pd.concat([data_positive_upsampled,data_negative]).reset_index(drop=True)
    return balanced_dataset

In [15]:
df_train, df_test = train_test_split(df, train_size = 0.8, shuffle = True)

In [16]:
df_train['Label'].value_counts()

0.0    2533
1.0    1556
Name: Label, dtype: int64

In [17]:
df_train = dataset_balance(df_train)
df_train['Label'].value_counts()

1.0    2533
0.0    2533
Name: Label, dtype: int64

In [18]:

tf_idf = TfidfVectorizer(ngram_range = (1,5))
tf_idf.fit(df['Comments'])
X_train = tf_idf.transform(df_train['cleanText'])
Y_train = df_train.Label
X_test = tf_idf.transform(df_test['cleanText'])
Y_test = df_test.Label

# TRAINING THE MACHINE LEARNING MODEL


1.   Logistic Regression
2.   Support Vector Machine
3.   Naive Bayes
4.   Random Forest
5.   Decision Tree



In [19]:
model_LR = LogisticRegression().fit(X_train,Y_train)

In [20]:
model_SVM = SVC(kernel='rbf', random_state = 42).fit(X_train, Y_train)

In [21]:
model_NB = ComplementNB().fit(X_train,Y_train)

In [22]:
model_RF = RandomForestClassifier(n_estimators= 250, criterion="entropy").fit(X_train, Y_train)

In [23]:
model_DC = DecisionTreeClassifier(criterion = 'entropy', random_state = 1).fit(X_train,Y_train)

In [24]:
LR_score = classification_report(Y_test, model_LR.predict(X_test))
print("LOGISTIC REGRESSION")
print(LR_score)

LOGISTIC REGRESSION
              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86       626
         1.0       0.80      0.74      0.77       397

    accuracy                           0.83      1023
   macro avg       0.82      0.81      0.81      1023
weighted avg       0.82      0.83      0.82      1023



In [25]:
SVC_score = classification_report(Y_test, model_SVM.predict(X_test))
print("SUPPORT VECTOR CLASSIFICATION")
print(SVC_score)

SUPPORT VECTOR CLASSIFICATION
              precision    recall  f1-score   support

         0.0       0.79      0.94      0.86       626
         1.0       0.87      0.61      0.71       397

    accuracy                           0.81      1023
   macro avg       0.83      0.77      0.79      1023
weighted avg       0.82      0.81      0.80      1023



In [26]:
NB_score = classification_report(Y_test, model_NB.predict(X_test))
print("NAIVE BAYES CLASSIFICATION")
print(NB_score)

NAIVE BAYES CLASSIFICATION
              precision    recall  f1-score   support

         0.0       0.89      0.77      0.83       626
         1.0       0.71      0.86      0.77       397

    accuracy                           0.81      1023
   macro avg       0.80      0.82      0.80      1023
weighted avg       0.82      0.81      0.81      1023



In [27]:
RF_score = classification_report(Y_test, model_RF.predict(X_test))
print("RANDOM FOREST CLASSIFICATION")
print(RF_score)

RANDOM FOREST CLASSIFICATION
              precision    recall  f1-score   support

         0.0       0.82      0.92      0.87       626
         1.0       0.85      0.68      0.76       397

    accuracy                           0.83      1023
   macro avg       0.83      0.80      0.81      1023
weighted avg       0.83      0.83      0.82      1023



In [28]:
DC_score = classification_report(Y_test, model_DC.predict(X_test))
print("DECISION TREE CLASSIFICATION")
print(DC_score)

DECISION TREE CLASSIFICATION
              precision    recall  f1-score   support

         0.0       0.79      0.86      0.83       626
         1.0       0.75      0.65      0.69       397

    accuracy                           0.78      1023
   macro avg       0.77      0.75      0.76      1023
weighted avg       0.78      0.78      0.77      1023



In [29]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import numpy as np 
import pandas as pd 
import re
import string
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras.layers import LSTM, GRU
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

In [31]:
# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [32]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [33]:
df.head()

,Unnamed: 0,Comments,Label,cleanText
0,0,আমি মনে করি যখন মেয়েরা উদ্দেশ্য অনুসারে বোবা ...,1.0,মেয়েরা উদ্দেশ্য বোবা বোবা আচরণ সুন্দর
1,1,আমি যখন কোনও গুরুত্বপূর্ণ প্রশ্ন জিজ্ঞাসা করি ...,1.0,গুরুত্বপূর্ণ প্রশ্ন পছন্দ আশা চালিয়ে পছন্দ
2,2,বাহ ... আমি দেখতে দেখতে পাচ্ছি সত্যিই দোষী দোষ...,1.0,বাহ পাচ্ছি দোষী দোষী সাব্যস্ত নোটপ্রেসড
3,3,"হাহ, মজার বিষয়।#আনোয়েড #গ্রুআপ",1.0,হাহ মজার বিষয় আনোয়েড গ্রুআপ
4,4,থান্ডার আমাকে 830 এ জেগে উঠলে এটি ভালবাসুন,1.0,থান্ডার জেগে উঠলে ভালবাসুন


In [34]:
df['Comments'].apply(lambda x:len(str(x).split())).max()

180

In [35]:
Y = pd.get_dummies(df['Label']).values
print('Shape of label tensor:', Y.shape)
X = df['cleanText']

Shape of label tensor: (5112, 2)


In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.8, random_state=42, shuffle = True)


In [37]:
maxlen = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [38]:
def create_embedding_matrix(filepath, word_idx, embedding_dim):
    vocab_size = len(word_idx) + 1  
    emb = np.zeros((vocab_size, embedding_dim))
    with open(filepath, 'r', encoding="utf8") as f:
        for i in f:
            x = i.split(' ')
            w, v = x[0], x[1:]
            if w in word_idx:
                emb[word_idx[w]] = np.asarray(v)[:embedding_dim]
    return emb

In [39]:
embedding_dim = 300
# embedding_matrix = create_embedding_matrix('/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/glove.840B.300d.txt', tokenizer.word_index, embedding_dim)
embedding_matrix = create_embedding_matrix('/content/drive/MyDrive/Bracu/Fall 2022/CSE 431/bn_glove.39M.300d.txt', tokenizer.word_index, embedding_dim)

In [40]:
# with tpu_strategy.scope():
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                          weights=[embedding_matrix], 
                          input_length=maxlen, 
                          trainable=True))
model.add(Bidirectional(LSTM(300,activation='tanh', dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(300,activation = 'relu'))
# model.add(Dropout(0.2))
model.add(Dense(300,activation = 'relu'))
# model.add(Dropout(0.2))
model.add(Dense(256,activation = 'relu'))
# model.add(Dropout(0.1))
# model.add(Dense(128,activation = 'relu'))
# # model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.BinaryAccuracy()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3299400   
                                                                 
 bidirectional (Bidirectiona  (None, 600)              1442400   
 l)                                                              
                                                                 
 dense (Dense)               (None, 300)               180300    
                                                                 
 dense_1 (Dense)             (None, 300)               90300     
                                                                 
 dense_2 (Dense)             (None, 256)               77056     
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                        

In [41]:
epochs = 100
batch_size = 512 #*tpu_strategy.num_replicas_in_sync
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001)])

Epoch 1/100
7/7 [==============================] - 33s 3s/step - loss: 0.6586 - binary_accuracy: 0.6068 - val_loss: 0.5324 - val_binary_accuracy: 0.6773
Epoch 2/100
7/7 [==============================] - 19s 3s/step - loss: 0.4538 - binary_accuracy: 0.7985 - val_loss: 0.3492 - val_binary_accuracy: 0.8521
Epoch 3/100
7/7 [==============================] - 19s 3s/step - loss: 0.2895 - binary_accuracy: 0.8841 - val_loss: 0.2799 - val_binary_accuracy: 0.8802
Epoch 4/100
7/7 [==============================] - 19s 3s/step - loss: 0.2042 - binary_accuracy: 0.9208 - val_loss: 0.2503 - val_binary_accuracy: 0.8888
Epoch 5/100
7/7 [==============================] - 19s 3s/step - loss: 0.1583 - binary_accuracy: 0.9392 - val_loss: 0.2583 - val_binary_accuracy: 0.8936
Epoch 6/100
7/7 [==============================] - 19s 3s/step - loss: 0.1139 - binary_accuracy: 0.9548 - val_loss: 0.2588 - val_binary_accuracy: 0.8875
Epoch 7/100
7/7 [==============================] - 18s 3s/step - loss: 0.0808 - bi

In [42]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

32/32 [==============================] - 8s 249ms/step - loss: 0.3472 - binary_accuracy: 0.8925
Test set
  Loss: 0.347
  Accuracy: 0.892


In [43]:
output = model.predict(X_test)


32/32 [==============================] - 7s 211ms/step


In [44]:
def flat(ndarray):
  array = []
  for x in ndarray:
    if x[0] > x[1]:
      array.append(0)
    else:
      array.append(1)
  return np.array(array)
  

In [45]:
flatOutput = flat(output)
print(flatOutput)
flatY = flat(Y_test)

[0 0 1 ... 0 0 0]


In [46]:
print(flatOutput.shape, flatY.shape)

(1023,) (1023,)


In [47]:
print(classification_report(flatY, flatOutput))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91       626
           1       0.86      0.86      0.86       397

    accuracy                           0.89      1023
   macro avg       0.89      0.89      0.89      1023
weighted avg       0.89      0.89      0.89      1023

